# Tutorial

cryoswath: python package, offering tools to swath process CryoSat-2 data.

**Scope:**  

Focused on mountain glaciers, where POCA fails.
Mainly users should eventually be able to calculate gridded dh/dt.

In [ ]:
# make the package available/from search path
import sys
sys.path.insert(0, "..")

## tl;dr

### jump to gridded product

To obtain gridded aggregates of point elevation estimates, first make
sure the dependencies are available. If you haven't done so, install the
packages listed in [`../requirements.txt`](../requirements.txt), and
download
[ArcticDEM mosaic at 100 m](https://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v4.1/100m/arcticdem_mosaic_100m_v4.1.tar.gz)
and extract the `*_dem.tif` to `../data/auxiliary/DEM/`. Then run the
following:

(The L1b data of ground tracks in the proximity of your region of
interest will be downloaded, processed, stored, aggregated, and finally
the product will be saved to disk).

In [ ]:
import cryoswath
# cryoswath.l3.build_dataset(region_of_interest="09-02", # any RGI o2region
#                            start_datetime="2014-07", # like yyyy-mm-dd with optional month and day
#                            end_datetime="2023-10",
#                         #    timestep_months = 1,
#                         #    window_ntimesteps = 3,
#                         #    spatial_res_meter=500,
#                            )

However, a best-practice is to first download the l1b data, then process
them to l2, and finally build the L3 dataset providing a dask cluster.
This is because the routines work more efficiently this way.

### download all L1b for your project

There is a wrapper to download all available L1b data for a certain
period and region. Say, you are interested in the changes on the West
Antartic Peninsula. The below will find the tracks passing over or close
by the region for the entire observational period and then download them
to data/auxiliary/L1b/<year>/<month> (specified in cryoswath.misc).  
(*You have to have the RGI files available if you pass a region code.
Alternatively, pass a polygon in EPSG:4326.*)

In [ ]:
from cryoswath import l1b

In [ ]:
# l1b.download_files(region_of_interest="19-20", # the region's RGI o2 code. o1 or glacier ids also work
#                    start_datetime="2010",
#                    end_datetime="2025",
#                    buffer_region_by=10_000, # also consider tracks in the proximity of 10 km
#                    )

Mind that L1b files are archived after some time (years). When they are,
their name changes. This can lead to confusion, when repeating steps as
downloading or processing.

## step-by-step

The step-by-step guide is work in progress. Feel free to have a sneak-peak into the

If you haven't done so, install the
packages listed in [`../requirements.txt`](../requirements.txt), and
download
[ArcticDEM mosaic at 100 m](https://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v4.1/100m/arcticdem_mosaic_100m_v4.1.tar.gz)
and extract the `*_dem.tif` to `../data/auxiliary/DEM/`. Further, you'll
probably need to download the RGI data (you'll be prompted) and you may
need to convert the esri files to feather (you can use the helper
function in module gis).

If you have custom demands, you may be interested to understand how you
can use the tools as you need them. Below, I will introduce some handy
once.

Lets get a list of CryoSat-2 overpasses for the Alps:

In [ ]:
from cryoswath.misc import *
from cryoswath import gis, l1b, l2, l3
import os
import geopandas as gpd

In [ ]:
o2_code = "11-01"
Alps_glaciers = load_o2region(o2_code)
all_cs_tracks = load_cs_ground_tracks()

In [ ]:
basin_rgi_id = find_region_id((75.85, 64.983333), scope="basin")

In [ ]:
basin_outlines = load_basins([basin_rgi_id]).unary_union

In [ ]:
example_tracks = all_cs_tracks[all_cs_tracks.intersects(
    gis.buffer_4326_shp(basin_outlines, 10_000))]
print("number of tracks:", example_tracks.shape[0], "\n last five:", example_tracks.tail(5))

In [ ]:
l1b_data = l1b.l1b_data.from_id(cs_time_to_id(example_tracks.iloc[0].name), drop_non_glacier_areas=False)

In [ ]:
import cryoswath.test_plots.waveform

In [ ]:
cryoswath.test_plots.waveform.coherence(l1b_data.isel(time_20_ku=[200]))

In [ ]:
cryoswath.test_plots.waveform.power(l1b_data.isel(time_20_ku=[200]))

In [ ]:
# trigger complete processing
l1b_data = l1b_data.append_best_fit_phase_index()

In [ ]:
cryoswath.test_plots.waveform.dem_transect(l1b_data.isel(time_20_ku=[200]))

It seems the range window was not aligned well. There is nothing you can
do about it. Choose another example.